# Train Pixel Level Annotation Model

### This notebook uses pixel level annotations to train a Random Forest Classifier to predict labels

We assume Pixel level annotations are available, as produced by the "../data/annotations/transform_polygon_annotations_to_pixels.ipynb" notebook. Feature selection and grid_search for optimal parameters has been done in a separate notebook ("Coepelduynen/make_train_model_on_annotations_coepelduynen.ipynb") and those outcomes are taken as given in this notebook.

Change the set Variables cell below as desired and then run the entire notebook to get cross_validation results as well as a final model trained on all data.

Date: 2024-01-12\
Author: Pieter Kouyzer, Michael de Winter

In [1]:
%load_ext autoreload
%autoreload 2

In [16]:
%matplotlib notebook
import pandas as pd
import geopandas as gpd
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import pprint
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score
from satellite_images_nso_datascience.training.train import train_imbalanced_model, cross_validation_balance_on_date
from satellite_images_nso_datascience.training.utils import get_cross_validation_results_filepath, get_model_filepath
from satellite_images_nso_datascience.training.metric_calculation import calculate_average_metrics, get_metrics
from sklearn.metrics import f1_score
from tif_model_iterator import tif_kernel_iterator
from filenames.file_name_generator import OutputFileNameGenerator
import glob
import contextlib
import io
import numpy as np
from satellite_images_nso_datascience.other import functions

data_dir = os.path.abspath(os.path.join(os.getcwd(), '..', '..')).replace("\\","/")+"/data/annotations/annotations_pixel_dataframes/"

In [17]:
# Set Variables
location = "Nieuwkoopse_plassen"
satellite_constellation = "Superview"

if location == "Voornes Duin":

    if satellite_constellation == "PNEO":
        # Check if file exists
        if not os.path.isfile(data_dir+"Voornes_Duin_PNEO_2024-01-29_pixel_annotations.parquet"):
            functions.download_file("https://e34a505986aa74678a5a0e0f.blob.core.windows.net/satellite-images-nso/Voornes_Duin/annotations_pixel_dataframes/2023_PNEO/Voornes_Duin_PNEO_2024-01-29_pixel_annotations.parquet", data_dir+"Voornes_Duin_PNEO_2024-01-29_pixel_annotations.parquet")

        annotated_pixels_filepath = data_dir+"Voornes_Duin_PNEO_2024-01-29_pixel_annotations.parquet"

    #TODO: Missing Superview here!


elif location == "Coepelduynen":
    if satellite_constellation == "PNEO":
        # Check if file exists
        if not os.path.isfile(data_dir+"annotaties_coepelduynen_to_pixel_2023.parquet"):
            functions.download_file("https://e34a505986aa74678a5a0e0f.blob.core.windows.net/satellite-images-nso/coepelduynen/annotations_pixel_dataframes/annotations_2023_PNEO/annotaties_coepelduynen_to_pixel_2023.parquet", data_dir+"annotaties_coepelduynen_to_pixel_2023.parquet")

        annotated_pixels_filepath = data_dir+"annotaties_coepelduynen_to_pixel_2023.parquet"
        df = pd.read_parquet(annotated_pixels_filepath)

    if satellite_constellation == "Superview":
        # Check if file exists
        if not os.path.isfile(data_dir+"annotaties_coepelduynen_to_pixel.csv"):
            functions.download_file("https://e34a505986aa74678a5a0e0f.blob.core.windows.net/satellite-images-nso/coepelduynen/annotations_pixel_dataframes/annotations_2019_2022_Superview/annotaties_coepelduynen_to_pixel.csv", data_dir+"annotaties_coepelduynen_to_pixel.csv")

        annotated_pixels_filepath = data_dir+"annotaties_coepelduynen_to_pixel.csv"
        df = pd.read_parquet(annotated_pixels_filepath)

   
elif location == "Schippersgat":
    if satellite_constellation == "PNEO":
        if not os.path.isfile(data_dir+"PNEO_waterplanten_annotations.parquet"):
            functions.download_file("https://e34a505986aa74678a5a0e0f.blob.core.windows.net/satellite-images-nso/Schippersgat/PNEO_waterplanten_annotations.parquet", data_dir+"PNEO_waterplanten_annotations.parquet")
        annotated_pixels_filepath = data_dir+"PNEO_waterplanten_annotations_schippersgat.parquet"
        df = pd.read_parquet(annotated_pixels_filepath)

    if satellite_constellation == "Superview":
        if not os.path.isfile(data_dir+"Superview_waterplanten_annotations.parquet"):
            functions.download_file("https://e34a505986aa74678a5a0e0f.blob.core.windows.net/satellite-images-nso/Schippersgat/Superview_waterplanten_annotations.parquet", data_dir+"Superview_waterplanten_annotations.parquet")
        annotated_pixels_filepath = data_dir+"Superview_waterplanten_annotations.parquet"
        df = pd.read_parquet(annotated_pixels_filepath)

elif location == "Nieuwkoopse_plassen":
    if satellite_constellation == "PNEO":
        if not os.path.isfile(data_dir+"PNEO_waterplanten_annotations.parquet"):
            functions.download_file("https://e34a505986aa74678a5a0e0f.blob.core.windows.net/satellite-images-nso/Schippersgat/PNEO_waterplanten_annotations.parquet", data_dir+"PNEO_waterplanten_annotations.parquet")
        annotated_pixels_filepath = data_dir+"PNEO_waterplanten_annotations_schippersgat.parquet"
        df = pd.read_parquet(annotated_pixels_filepath)


        df = pd.concat([df, pd.read_parquet(data_dir+"PNEO_waterplanten_annotations_Nieuwkoopse_Plassen.parquet")])

    if satellite_constellation == "Superview":
        
        if not os.path.isfile(data_dir+"Superview_waterplanten_annotations.parquet"):
            print("Downloading file")
            functions.download_file("https://e34a505986aa74678a5a0e0f.blob.core.windows.net/satellite-images-nso/Nieuwkoopse_Plassen/Superview_waterplanten_annotations.parquet", data_dir+"Superview_waterplanten_annotations.parquet")
        annotated_pixels_filepath = data_dir+"Superview_waterplanten_annotations.parquet"
        df = pd.read_parquet(annotated_pixels_filepath)

        #TODO: VARY STRANGE REASON THAT NDWI AND NDVI GET FLIPPED WHILE READING IN DATA!!!!!!!!!!!!!!!!!!!!
        df['temp_ndwi'] = df['ndvi']
        df['ndvi'] =  df['ndwi']
        df['ndwi'] = df['temp_ndwi']
        df = df.drop(['temp_ndwi'], axis=1)


In [18]:
# Optimal parameters and features Voornes
if location == "Voornes Duin":
    selected_features = ['r', 'g', 'b', 'n', 'e', 'd', 'ndvi','re_ndvi', 'height']
    optimal_parameters = {
        "n_estimators": 10, 
        "min_samples_split": 5, 
        "min_samples_leaf": 1,
        "max_features": "auto", 
        "bootstrap": False
    }

elif location == "Coepelduynen":
    # Optimal parameters and features
    selected_features = ['r','g','b','n','e','d','ndvi','re_ndvi']
    optimal_parameters = {
        "n_estimators": 10, 
        "min_samples_split": 5, 
        "min_samples_leaf": 1,
        "bootstrap": False
    }

elif location == "Schippersgat" and satellite_constellation == "Superview":
    # Optimal parameters and features
    selected_features = ['r', 'g', 'b', 'i', 'ndvi', 'height']
    optimal_parameters = {
        "n_estimators": 10, 
        "min_samples_split": 5, 
        "min_samples_leaf": 1,
        "bootstrap": False
    }

elif location == "Schippersgat" and satellite_constellation == "PNEO":
    # Optimal parameters and features
    selected_features = ['r', 'g', 'b', 'n', 'e', 'd','ndvi', 're_ndvi']
    optimal_parameters = {
        "n_estimators": 10, 
        "min_samples_split": 5, 
        "min_samples_leaf": 1,
        "bootstrap": False
    }


elif location == "Nieuwkoopse_plassen" and satellite_constellation == "PNEO":
    # Optimal parameters and features
    selected_features = ['r', 'g', 'b', 'n', 'e', 'd','ndvi', 're_ndvi']
    optimal_parameters = {
        "n_estimators": 10, 
        "min_samples_split": 5, 
        "min_samples_leaf": 1,
        "bootstrap": False
    }

elif location == "Nieuwkoopse_plassen" and satellite_constellation == "Superview":
    # Optimal parameters and features
    selected_features = ['r', 'g', 'b', 'i', 'ndvi', 'ndwi']
    optimal_parameters = {
        "n_estimators": 1, 
        "min_samples_split": 2, 
        "min_samples_leaf": 2,
        "bootstrap": True
    }

In [19]:
df['label'] = df['label'].str.replace("GRound","Ground" )

In [20]:
# This is to give an indication of the amount of data points per label
df['label'].value_counts()

label
Water          3814785
Ground         2542204
Waterplants     235613
Name: count, dtype: int64

In [20]:
# Downsample Water if necessary
#df = pd.concat([df[df["label"] == "Water"].sample(1719330, random_state=1), df[df["label"] == "Ground"], df[df["label"] == "Waterplants"]])

### Cross Validation

We do cross-validation, where the folds are decided by the 'date' column. This is to avoid pixels from the same image from ending up in both the train and test datasets. We display the metrics averaged over the folds and write the results to a pickle.

In [21]:
model = RandomForestClassifier(**optimal_parameters)
scaler = StandardScaler()

In [22]:
if location == "Voornes Duin":
    cv =5 
    random_state = 1337
    sampling_type_boundary = 100000
elif location == "Coepelduynen":
    cv =4 
    random_state = 1337
    sampling_type_boundary = 898609
elif location == "Schippersgat":
    cv =2
    random_state = 1337
    sampling_type_boundary = 898609

elif location == "Nieuwkoopse_plassen":
    cv = 7
    random_state = 1337
    sampling_type_boundary = 898609

In [23]:
df['date'] = df['date'].str.split('_').str[0]

In [9]:
results = cross_validation_balance_on_date(data=df, model=model, cv=cv, features=selected_features, random_state=random_state, sampling_type_boundary=sampling_type_boundary , scaler=scaler)

---------fold: 1
Picked hold out dates: 
['20190302', '20190629']
Oversampling to rebalance dataset


KeyboardInterrupt: 

In [ ]:
calculate_average_metrics(results=results)

,precision,recall,f1-score
Ground,0.888229,0.892027,0.878536
Water,0.936239,0.912472,0.907345
Waterplants,0.551228,0.499053,0.414979


In [75]:
cross_validation_results_filepath = get_cross_validation_results_filepath(location=location, satellite_constellation=satellite_constellation, df=df)
print(f"Saving to {cross_validation_results_filepath}")
with open(cross_validation_results_filepath, "wb") as file:
    pickle.dump(results, file)

Saving to ../saved_models/Superview_Nieuwkoopse_plassen_20190629_to_20221012_cross_validation_results.pkl


## Cross validation randomised

In [13]:
# Initialize KFold
kf = KFold(n_splits=4, shuffle=True, random_state=42)

scores = cross_val_score(model, df[selected_features], df["label"], cv=kf)

2024/06/19 16:17:51 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '84ba19211c724bb09499c97fb4262da7', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2024/06/19 16:17:51 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\ProgramData\Anaconda3\envs\py310\envs\py312\Lib\site-packages\mlflow\data\digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead."


<IPython.core.display.Javascript object>

2024/06/19 16:24:52 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '75db5e3b677c4ac284950c28aab87e03', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2024/06/19 16:24:52 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\ProgramData\Anaconda3\envs\py310\envs\py312\Lib\site-packages\mlflow\data\digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead."


<IPython.core.display.Javascript object>

2024/06/19 16:32:10 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '60b74d828b764dab85a3e5bd0f6fa6c3', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2024/06/19 16:32:10 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\ProgramData\Anaconda3\envs\py310\envs\py312\Lib\site-packages\mlflow\data\digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead."


<IPython.core.display.Javascript object>

2024/06/19 16:39:14 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '6a7b8a2bcf8e44e7a6f1b6fb8bdfc166', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2024/06/19 16:39:15 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\ProgramData\Anaconda3\envs\py310\envs\py312\Lib\site-packages\mlflow\data\digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead."


<IPython.core.display.Javascript object>

In [14]:
scores

array([0.9962408 , 0.996042  , 0.99603984, 0.99614319])

### Try some unbalanced models

In [24]:
unbalanced_model = RandomForestClassifier(**optimal_parameters)
unblanced_final_scaler = StandardScaler()

In [25]:
unbalanced_model.fit(unblanced_final_scaler.fit_transform(df[selected_features]), df['label'])

2024/08/08 09:49:12 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'ba6e31746b804225b9569fffcb916417', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2024/08/08 09:49:12 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


<IPython.core.display.Javascript object>

RandomForestClassifier(min_samples_leaf=2, n_estimators=1)

In [26]:
f1_score(unbalanced_model.predict(unblanced_final_scaler.transform(df[selected_features])), df["label"], average=None)

array([0.99530845, 0.997549  , 0.98736639])

In [27]:
final_artefact = {
    "model": unbalanced_model,
    "scaler": unblanced_final_scaler
}

### Export Definitive model.

Trains a Random Forest Classifier model on all data and writes it to a pickle file for later use. This is the definitive model output by this notebook.

In [28]:
final_model = RandomForestClassifier(**optimal_parameters)
final_scaler = StandardScaler()

train_imbalanced_model(
    X_train=df[selected_features], 
    y_train=df["label"], 
    model=final_model, 
    random_state=1337, 
    sampling_type_boundary=sampling_type_boundary ,
    scaler=final_scaler
)
pprint.pprint(get_metrics(y=df["label"], X=df[selected_features], model=final_model, scaler=final_scaler))

Oversampling to rebalance dataset


2024/08/08 11:00:30 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'e7a59748ee0b4530b35a7e467fbc95df', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2024/08/08 11:00:43 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were not specified. To automatically record training metrics, provide training labels as inputs to the model training function.
2024/08/08 11:00:43 WARNING mlflow.sklearn: Failed to infer model signature: the trained model does not have a `predict` or `transform` function, which is required in order to infer the signature
2024/08/08 11:00:44 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2024/08/08 11:01:48 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '3f3153cc26a5411f9be6eb6230e45887', which will track hyperparameters, performance metrics, model

Fitting model


2024/08/08 11:02:00 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


<IPython.core.display.Javascript object>

{'Ground': {'f1-score': 0.9957245006692077,
            'precision': 0.9951472503639267,
            'recall': 0.9963024210488222,
            'support': 2542204},
 'Water': {'f1-score': 0.9975881134593813,
           'precision': 0.9981121979982581,
           'recall': 0.997064578999865,
           'support': 3814785},
 'Waterplants': {'f1-score': 0.9916384171220826,
                 'precision': 0.9894323645982102,
                 'recall': 0.9938543289207302,
                 'support': 235613}}


In [29]:
final_artefact = {
    "model": final_model,
    "scaler": final_scaler
}

In [37]:
location

'Nieuwkoopse_plassen'

In [38]:
final_model_filepath =get_model_filepath(location=location, satellite_constellation=satellite_constellation, df=df)
print(f"Saving to {final_model_filepath}")
with open(final_model_filepath, "wb") as file:
    pickle.dump(final_artefact, file)

Saving to C:/repos/satellite-images-nso-datascience//saved_models/Superview_Nieuwkoopse_plassen_20190302_to_20221012_random_forest_classifier_test.sav


# Model tests

Look in the tests/annotations_model_tests/test_annotation_model_nieuwkoopse_plassen.py file for all the tests and run with pytests

# NDWI and NDVI do not match, somehow gets twisted, do not forget this!

In [86]:
df[(df["rd_y"]  == 463906.25) & (df["rd_x"] == 117219.25)][selected_features]

,r,g,b,i,ndvi,ndwi
231,646.0,458.0,358.0,273.0,125.0,59.0


In [87]:
df_annotated_export_data[(df_annotated_export_data["rd_y"]  == 463906.25) & (df_annotated_export_data["rd_x"] == 117219.25)][selected_features]

,r,g,b,i,ndvi,ndwi
9982,646.0,458.0,358.0,273.0,59.0,125.0


In [80]:
final_model.predict(final_scaler.transform(df_annotated_export_data[(df_annotated_export_data["rd_y"]  == 463906.25) & (df_annotated_export_data["rd_x"] == 117219.25)][selected_features]))

array(['Water'], dtype=object)